# Geração de Imagens com VAE Pré-Treinado (CPU)
Este notebook demonstra como usar um modelo de **Autoencoder Variacional (VAE)** já treinado, especificamente o `AutoencoderKL` utilizado no **Stable Diffusion**, para gerar imagens a partir de vetores latentes.

O modelo utilizado foi publicado pela equipe da Stability AI e é acessível através da biblioteca `diffusers`.

A execução será feita na **CPU**, para garantir compatibilidade com ambientes sem GPU.

## O que são as bibliotecas utilizadas?
- `torch` (PyTorch): Biblioteca de machine learning para criação e execução de redes neurais.
- `torchvision`: Biblioteca complementar ao PyTorch, usada para manipular imagens, carregar datasets e gerar grids de imagens.
- `diffusers`: Biblioteca da Hugging Face que oferece acesso fácil a modelos de geração de imagens como diffusion models, VAEs e outros.
- `accelerate`: Backend utilizado pela `diffusers` para configurar o ambiente de execução (GPU/CPU, paralelismo, etc).
- `matplotlib`: Biblioteca de visualização usada para exibir as imagens.

In [ ]:
!pip install diffusers accelerate torch torchvision --quiet

In [ ]:
from diffusers import AutoencoderKL
import torch
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

## O que é um AutoencoderKL?
- É uma variante de Autoencoder Variacional usada para codificar imagens em um espaço latente de distribuição normal.
- `AutoencoderKL` é a implementação do VAE utilizado internamente pelo modelo **Stable Diffusion**.
- Ao contrário de um autoencoder clássico, ele aprende não apenas um ponto, mas uma **distribuição de vetores latentes**.
- Esse modelo foi treinado para reconstruir imagens com alta fidelidade a partir desses vetores.


In [ ]:
# Carrega o modelo VAE na CPU
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=torch.float32)
vae = vae.to("cpu")
vae.eval()

## Geração de imagens com vetores aleatórios no espaço latente
Criamos vetores `z` com distribuição normal padrão (ruído), e os passamos pelo decoder do VAE para gerar imagens.
O espaço latente tem dimensão `[batch, 4, 32, 32]`, ou seja, 4 canais com 32x32 cada.

In [ ]:
batch = 8
z = torch.randn(batch, 4, 32, 32)
with torch.no_grad():
    imgs = vae.decode(z).sample
imgs = (imgs * 0.5 + 0.5).clamp(0,1)
grid = make_grid(imgs, nrow=4)
plt.figure(figsize=(8,4))
plt.imshow(grid.permute(1,2,0))
plt.axis('off')
plt.title("Imagens geradas com VAE pré-treinado (CPU)")
plt.show()

## Manipulação direta do vetor latente
Aqui criamos um vetor `z` com todos os valores zerados e ativamos um ponto específico com valor alto (`3.0`).
Isso nos permite explorar como partes específicas do vetor latente afetam a imagem gerada.

In [ ]:
z = torch.zeros(batch, 4, 32, 32)
z[:, :, 16, 16] = 3.0
with torch.no_grad():
    imgs = vae.decode(z).sample
imgs = (imgs * 0.5 + 0.5).clamp(0,1)
grid = make_grid(imgs, nrow=4)
plt.figure(figsize=(8,4))
plt.imshow(grid.permute(1,2,0))
plt.axis('off')
plt.title("Imagens com ponto latente [16,16] ativado")
plt.show()

## Interpolação entre dois vetores latentes
Interpolamos entre dois vetores `z0` e `z1` com pesos crescentes, e observamos a transição visual entre eles.
Esse tipo de operação revela como o espaço latente é **suave** e permite **variações contínuas** nas imagens.

In [ ]:
z0 = torch.randn(1, 4, 32, 32)
z1 = torch.randn(1, 4, 32, 32)
steps = 8
interpolated = []
for alpha in torch.linspace(0, 1, steps):
    z_interp = (1 - alpha) * z0 + alpha * z1
    with torch.no_grad():
        img = vae.decode(z_interp).sample
        interpolated.append((img * 0.5 + 0.5).clamp(0, 1))
imgs_interp = torch.cat(interpolated, dim=0)
grid = make_grid(imgs_interp, nrow=steps)
plt.figure(figsize=(16,4))
plt.imshow(grid.permute(1,2,0))
plt.axis('off')
plt.title("Interpolação no espaço latente entre dois vetores z")
plt.show()